In [1]:
import io
import sys
PATH = "D:/OneDrive - Universidad Tecnológica de Bolívar/Apps/NaturalLanguageProcessing-MING-2903/"
DIR_DATA = PATH + "data/input/"
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import spacy
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
from sklearn import svm
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import preprocessing
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder 
from logic.text_processing import TextProcessing
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, precision_score

In [2]:
tp = TextProcessing(lang='es')
le = LabelEncoder()
nlp = spacy.load("es_core_news_sm")

Language: Text Processing
es: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [3]:
data_train = pd.read_csv(DIR_DATA + 'tass2018_es_train.csv', sep=',')
data_train

,tweetid,user,content,date,lang,sentiment/polarity/value
0,768213876278165504,OnceBukowski,-Me caes muy bien \r\n-Tienes que jugar más pa...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\r\n\r\...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N
...,...,...,...,...,...,...
1003,814846333601320960,jdomperfects,Para mí mi mejor año fue el 2015,2016-12-30 14:51:19,es,P
1004,813731371076243461,GonzaloRoman_91,Hoy va a ser un dia muy largo...,2016-12-27 13:00:51,es,N
1005,818399956792905728,vaneftdallas,11:11 que no me encuentre mal,2017-01-09 10:12:09,es,NEU
1006,815715581878009858,carmen_ligero,Es que son muy bonitas jo,2017-01-02 00:25:24,es,P


In [4]:
data_test = pd.read_csv(DIR_DATA + 'tass2018_es_development.csv', sep=',')
data_test

,tweetid,user,content,date,lang,sentiment/polarity/value
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,2016-08-31 13:28:49,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,2016-08-31 21:08:54,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",2016-08-31 21:07:40,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,2016-08-31 21:07:30,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,2016-08-31 21:15:56,es,N
...,...,...,...,...,...,...
501,757593585634213888,RubenRuCh,@Davorias Pero es que está muy rica,2016-07-25 15:09:20,es,P
502,757585725764362240,rayjaken,@narvalillo Eso es antiguo ya,2016-07-25 14:38:06,es,N
503,757097408304148480,Fuhyo__,Mi TL esta vacía a estas horas,2016-07-24 06:17:42,es,N
504,757971881077182464,xLytherinDovah,"Ya tengo uno, pero ese es más cheto @aweamasome.",2016-07-26 16:12:33,es,NEU


In [5]:
x_train = [tp.transformer(row) for row in data_train['content'].tolist()]
#y_train = le.fit_transform(data_train['sentiment/polarity/value'])
y_train = data_train['sentiment/polarity/value']
len(x_train), len(y_train)

(1008, 1008)

In [6]:
x_test = [tp.transformer(row) for row in data_test['content'].tolist()]
#y_test = le.fit_transform(data_test['sentiment/polarity/value'])
y_test = data_test['sentiment/polarity/value']
len(x_test), len(y_test)

(506, 506)

In [7]:
for row in x_train:
    print(row)
    doc = nlp(str(row))
    pos = [(token.text, token.pos_) for token in doc]
    print(pos)

me caes muy bien tienes que jugar mas partidas al lol con russel y conmigo por que tan otako deja de ser otako haber si me muero
[('me', 'PRON'), ('caes', 'VERB'), ('muy', 'ADV'), ('bien', 'ADV'), ('tienes', 'VERB'), ('que', 'SCONJ'), ('jugar', 'VERB'), ('mas', 'PROPN'), ('partidas', 'NOUN'), ('al', 'ADP'), ('lol', 'NOUN'), ('con', 'ADP'), ('russel', 'PROPN'), ('y', 'CCONJ'), ('conmigo', 'NOUN'), ('por', 'ADP'), ('que', 'SCONJ'), ('tan', 'ADV'), ('otako', 'VERB'), ('deja', 'VERB'), ('de', 'ADP'), ('ser', 'AUX'), ('otako', 'VERB'), ('haber', 'AUX'), ('si', 'SCONJ'), ('me', 'PRON'), ('muero', 'VERB')]
mention a. que puto mal escribo b. me sigo surrando help 3. ha quedado raro el cometelo ahi jajajaja
[('mention', 'PROPN'), ('a.', 'PROPN'), ('que', 'PRON'), ('puto', 'ADJ'), ('mal', 'ADV'), ('escribo', 'NOUN'), ('b.', 'PROPN'), ('me', 'PRON'), ('sigo', 'VERB'), ('surrando', 'VERB'), ('help', 'PROPN'), ('3', 'NUM'), ('.', 'PUNCT'), ('ha', 'AUX'), ('quedado', 'VERB'), ('raro', 'ADJ'), ('el',

[('tengo', 'VERB'), ('una', 'DET'), ('adiccion', 'NOUN'), ('muy', 'ADV'), ('seria', 'ADJ'), ('al', 'ADP'), ('burguer', 'NOUN'), ('king', 'ADJ'), ('necesito', 'VERB'), ('ayuda', 'NOUN'), ('profesional', 'ADJ')]
me dice un pajarito que ahora mismo no puede reservar en la web de bibo lo nuevo de mention. espero que sea por la demanda.
[('me', 'PRON'), ('dice', 'VERB'), ('un', 'DET'), ('pajarito', 'NOUN'), ('que', 'PRON'), ('ahora', 'ADV'), ('mismo', 'ADJ'), ('no', 'ADV'), ('puede', 'AUX'), ('reservar', 'VERB'), ('en', 'ADP'), ('la', 'DET'), ('web', 'NOUN'), ('de', 'ADP'), ('bibo', 'NOUN'), ('lo', 'PRON'), ('nuevo', 'ADJ'), ('de', 'ADP'), ('mention', 'PROPN'), ('.', 'PUNCT'), ('espero', 'VERB'), ('que', 'SCONJ'), ('sea', 'AUX'), ('por', 'ADP'), ('la', 'DET'), ('demanda', 'NOUN'), ('.', 'PUNCT')]
mention a ti por responder espero y deseo que te salgan nuevos proyectos besitos
[('mention', 'NOUN'), ('a', 'ADP'), ('ti', 'PRON'), ('por', 'ADP'), ('responder', 'VERB'), ('espero', 'VERB'), ('y',

[('mention', 'PROPN'), ('pero', 'CCONJ'), ('explicar', 'VERB'), ('a', 'ADP'), ('fondo', 'NOUN'), ('las', 'DET'), ('zonas', 'NOUN'), ('habitables', 'ADJ'), ('y', 'CCONJ'), ('que', 'SCONJ'), ('condiciones', 'NOUN'), ('necesita', 'VERB'), ('un', 'DET'), ('planeta', 'NOUN'), ('para', 'ADP'), ('serlo', 'AUX')]
mention no se si hubo mas gente pero ayer te taguee yo ayy hannibal es que moooooola tanto
[('mention', 'PROPN'), ('no', 'ADV'), ('se', 'PRON'), ('si', 'SCONJ'), ('hubo', 'AUX'), ('mas', 'PROPN'), ('gente', 'NOUN'), ('pero', 'CCONJ'), ('ayer', 'ADV'), ('te', 'PRON'), ('taguee', 'VERB'), ('yo', 'PRON'), ('ayy', 'PROPN'), ('hannibal', 'PROPN'), ('es', 'AUX'), ('que', 'SCONJ'), ('moooooola', 'VERB'), ('tanto', 'ADV')]
hastag inhabilitado hastag mujeres tapadas hasta los ojos hastag teneis un sentido inamovible del mundo y de la vida.
[('hastag', 'VERB'), ('inhabilitado', 'ADJ'), ('hastag', 'VERB'), ('mujeres', 'NOUN'), ('tapadas', 'ADJ'), ('hasta', 'ADP'), ('los', 'DET'), ('ojos', 'NOUN'

[('mention', 'PROPN'), ('.', 'PUNCT'), ('estas', 'DET'), ('palabras', 'NOUN'), ('del', 'ADP'), ('poema', 'NOUN'), ('instantes', 'ADJ'), ('son', 'AUX'), ('de', 'ADP'), ('nadine', 'NOUN'), ('stair', 'NOUN'), ('.', 'PUNCT'), ('escritora', 'NOUN'), ('norteamericana', 'ADJ'), ('a', 'ADP'), ('la', 'DET'), ('q', 'NOUN'), ('le', 'PRON'), ('gustan', 'VERB'), ('los', 'DET'), ('helados', 'NOUN'), ('.', 'PUNCT')]
preparandome un colacao pa olvidarme de aquel bacalao...
[('preparandome', 'VERB'), ('un', 'DET'), ('colacao', 'NOUN'), ('pa', 'ADP'), ('olvidarme', 'VERB'), ('de', 'ADP'), ('aquel', 'DET'), ('bacalao', 'PROPN'), ('...', 'PUNCT')]
gracioso bc en la pelicula de nunca apagues la luz utilizan luz negra y hoy he llegado a casa y han puesto uno en el bano
[('gracioso', 'ADJ'), ('bc', 'PROPN'), ('en', 'ADP'), ('la', 'DET'), ('pelicula', 'NOUN'), ('de', 'ADP'), ('nunca', 'ADV'), ('apagues', 'NOUN'), ('la', 'DET'), ('luz', 'NOUN'), ('utilizan', 'VERB'), ('luz', 'NOUN'), ('negra', 'ADJ'), ('y', 'C

[('mention', 'NOUN'), ('totalmente', 'ADV'), ('pobre', 'ADJ'), ('gappy', 'NOUN'), ('...', 'PUNCT'), ('que', 'PRON'), ('inocente', 'ADJ'), ('es', 'AUX'), ('no', 'ADV'), ('sabe', 'VERB'), ('con', 'ADP'), ('quien', 'PRON'), ('se', 'PRON'), ('ha', 'AUX'), ('encontrado', 'VERB'), ('.', 'PUNCT')]
mention 2. hace mucho tiempo hablabamos por skype por un amigo en comun y estaba guay ademas de que eres muy mona
[('mention', 'PROPN'), ('2', 'NUM'), ('.', 'PUNCT'), ('hace', 'VERB'), ('mucho', 'DET'), ('tiempo', 'NOUN'), ('hablabamos', 'VERB'), ('por', 'ADP'), ('skype', 'NOUN'), ('por', 'ADP'), ('un', 'DET'), ('amigo', 'NOUN'), ('en', 'ADP'), ('comun', 'NOUN'), ('y', 'CCONJ'), ('estaba', 'AUX'), ('guay', 'VERB'), ('ademas', 'NOUN'), ('de', 'ADP'), ('que', 'SCONJ'), ('eres', 'AUX'), ('muy', 'ADV'), ('mona', 'ADJ')]
buen partido a todos mention suerte en la final de losers y a ver si nos volvemos a ver en la gran final. saludo a esos casters
[('buen', 'ADJ'), ('partido', 'NOUN'), ('a', 'ADP'), ('tod

...

[('las', 'DET'), ('palmas', 'NOUN'), ('nuevo', 'ADJ'), ('y', 'CCONJ'), ('modesto', 'ADJ'), ('lider', 'NOUN'), ('de', 'ADP'), ('division', 'NOUN'), ('empieza', 'VERB'), ('por', 'ADP'), ('l', 'PROPN'), ('como', 'SCONJ'), ('leicester', 'NOUN'), ('...', 'PUNCT'), ('hastag', 'VERB')]
y en nada estara disponible via hastag mi nueva sesion con el mejor house y un rollazo flipante
[('y', 'CCONJ'), ('en', 'ADP'), ('nada', 'NOUN'), ('estara', 'VERB'), ('disponible', 'ADJ'), ('via', 'NOUN'), ('hastag', 'VERB'), ('mi', 'DET'), ('nueva', 'ADJ'), ('sesion', 'NOUN'), ('con', 'ADP'), ('el', 'DET'), ('mejor', 'ADJ'), ('house', 'NOUN'), ('y', 'CCONJ'), ('un', 'DET'), ('rollazo', 'NOUN'), ('flipante', 'ADV')]
mention estoy recuperandome del video y el momento bird shit parece una parodia de si mismo
[('mention', 'PROPN'), ('estoy', 'AUX'), ('recuperandome', 'ADJ'), ('del', 'ADP'), ('video', 'NOUN'), ('y', 'CCONJ'), ('el', 'DET'), ('momento', 'NOUN'), ('bird', 'PROPN'), ('shit', 'PROPN'), ('parece', 'VERB

[('hoy', 'ADV'), ('ha', 'AUX'), ('sido', 'AUX'), ('ver', 'VERB'), ('a', 'ADP'), ('un', 'DET'), ('perro', 'NOUN'), ('muerto', 'ADJ'), ('en', 'ADP'), ('el', 'DET'), ('anden', 'NOUN'), ('izdo', 'ADJ'), ('.', 'PUNCT'), ('de', 'ADP'), ('la', 'DET'), ('autopista', 'NOUN'), ('huelvasevilla', 'ADJ'), ('y', 'CCONJ'), ('entrarme', 'VERB'), ('toda', 'DET'), ('la', 'DET'), ('impotencia', 'NOUN'), ('del', 'ADP'), ('mundo', 'NOUN')]
.mention mention vamos que usted es el portavoz de millones de personas. una inmensa... minoria. le molesta la minorizacion
[('.mention', 'NUM'), ('mention', 'PROPN'), ('vamos', 'VERB'), ('que', 'SCONJ'), ('usted', 'PRON'), ('es', 'AUX'), ('el', 'DET'), ('portavoz', 'NOUN'), ('de', 'ADP'), ('millones', 'NOUN'), ('de', 'ADP'), ('personas', 'NOUN'), ('.', 'PUNCT'), ('una', 'DET'), ('inmensa', 'ADJ'), ('...', 'PUNCT'), ('minoria', 'ADJ'), ('.', 'PUNCT'), ('le', 'PRON'), ('molesta', 'VERB'), ('la', 'DET'), ('minorizacion', 'PROPN')]
mention hola tomas habeis visto los nuevos

[('mention', 'PROPN'), ('no', 'ADV'), ('estoy', 'AUX'), ('de', 'ADP'), ('acuerd', 'VERB'), ('retwitteo', 'NOUN'), ('cuando', 'SCONJ'), ('creo', 'VERB'), ('conveniente', 'ADJ'), ('pero', 'CCONJ'), ('aporto', 'VERB'), ('mi', 'DET'), ('opinion', 'NOUN'), ('citando', 'VERB'), ('y', 'CCONJ'), ('divulgando', 'VERB'), ('y', 'CCONJ'), ('asi', 'ADV'), ('seguire', 'ADJ'), ('mention', 'PROPN')]
hola mention una pregunta la visita a la fabrica es gratuita o hay que pagar algo un saludo bonicos y bonicas mios
[('hola', 'PROPN'), ('mention', 'PROPN'), ('una', 'DET'), ('pregunta', 'NOUN'), ('la', 'DET'), ('visita', 'NOUN'), ('a', 'ADP'), ('la', 'DET'), ('fabrica', 'NOUN'), ('es', 'AUX'), ('gratuita', 'ADJ'), ('o', 'CCONJ'), ('hay', 'AUX'), ('que', 'SCONJ'), ('pagar', 'VERB'), ('algo', 'PRON'), ('un', 'DET'), ('saludo', 'NOUN'), ('bonicos', 'ADJ'), ('y', 'CCONJ'), ('bonicas', 'ADJ'), ('mios', 'NOUN')]
yo mantendria que toby esta vivo si no fuese por el tuit de keegs por el insta de troian de hace unos

[('la', 'DET'), ('vida', 'NOUN'), ('me', 'PRON'), ('tiene', 'VERB'), ('muy', 'ADV'), ('confundida', 'ADJ')]
mention pero pero porque eres muy guapa
[('mention', 'NOUN'), ('pero', 'CCONJ'), ('pero', 'CCONJ'), ('porque', 'SCONJ'), ('eres', 'AUX'), ('muy', 'ADV'), ('guapa', 'ADJ')]
mention pero eso no es abstracto
[('mention', 'PROPN'), ('pero', 'CCONJ'), ('eso', 'PRON'), ('no', 'ADV'), ('es', 'AUX'), ('abstracto', 'NOUN')]
mention ya ya pero aun asi no me convencio
[('mention', 'PROPN'), ('ya', 'ADV'), ('ya', 'ADV'), ('pero', 'CCONJ'), ('aun', 'ADV'), ('asi', 'ADV'), ('no', 'ADV'), ('me', 'PRON'), ('convencio', 'VERB')]
mention es que no ya estoy cansado
[('mention', 'PROPN'), ('es', 'AUX'), ('que', 'SCONJ'), ('no', 'ADV'), ('ya', 'ADV'), ('estoy', 'AUX'), ('cansado', 'ADJ')]
las mejores fotos son las que no se ven
[('las', 'DET'), ('mejores', 'ADJ'), ('fotos', 'NOUN'), ('son', 'AUX'), ('las', 'DET'), ('que', 'PRON'), ('no', 'ADV'), ('se', 'PRON'), ('ven', 'VERB')]
la verdad es que hoy t

[('mention', 'PROPN'), ('no', 'ADV'), ('pero', 'CCONJ'), ('a', 'ADP'), ('las', 'DET'), ('si', 'SCONJ')]
no me habeis curado el sad
[('no', 'ADV'), ('me', 'PRON'), ('habeis', 'VERB'), ('curado', 'VERB'), ('el', 'DET'), ('sad', 'NOUN')]
es que me lo como de lo bonito que es
[('es', 'AUX'), ('que', 'SCONJ'), ('me', 'PRON'), ('lo', 'PRON'), ('como', 'SCONJ'), ('de', 'ADP'), ('lo', 'PRON'), ('bonito', 'ADJ'), ('que', 'SCONJ'), ('es', 'AUX')]
mention a mi a las que son las no too bad
[('mention', 'NOUN'), ('a', 'ADP'), ('mi', 'PROPN'), ('a', 'ADP'), ('las', 'DET'), ('que', 'PRON'), ('son', 'AUX'), ('las', 'DET'), ('no', 'ADV'), ('too', 'PROPN'), ('bad', 'PRON')]
tio el final de sao
[('tio', 'VERB'), ('el', 'DET'), ('final', 'NOUN'), ('de', 'ADP'), ('sao', 'NOUN')]
mention ya se que es feo pero
[('mention', 'PROPN'), ('ya', 'ADV'), ('se', 'PRON'), ('que', 'SCONJ'), ('es', 'AUX'), ('feo', 'ADJ'), ('pero', 'CCONJ')]
dia poco productivo el de hoy
[('dia', 'ADP'), ('poco', 'ADV'), ('productivo', 

In [8]:
class POSExtraction(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, list_messages):
        try:
            result = self.get_features(list_messages)
            return result
        except Exception as e:
            print('Error transform: {0}'.format(e))
            
    def get_features(self, list_messages):
        result = {}
        i = 0 
        for row in list_messages:
            dict_pos = {}
            doc = nlp(str(row))
            pos = [token.pos_ for token in doc]
            for token in pos:
                if token not in dict_pos:
                    dict_pos[token] = 1
                else:
                    val = dict_pos[token]
                    dict_pos[token] = val + 1
            result[i] = dict_pos
            i += 1
        features = pd.DataFrame.from_dict(result, orient='index').fillna(0)
        return features.to_numpy()

In [9]:
bow_vector = CountVectorizer(analyzer='word', ngram_range=(1, 3))
pos_vector = POSExtraction()

In [10]:
pos_vector.transform(['Hola esto es una prueba', 'otra prueba'])

array([[1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [11]:
preprocessor = FeatureUnion([
    ('bow_vector', bow_vector),
    ('pos_vector', pos_vector)
])

In [12]:
preprocessor.fit(x_train)

FeatureUnion(transformer_list=[('bow_vector',
                                CountVectorizer(ngram_range=(1, 3))),
                               ('pos_vector', POSExtraction())])

In [13]:
x_train = preprocessor.transform(x_train)
x_train

<1008x26012 sparse matrix of type '<class 'numpy.float64'>'
	with 47464 stored elements in Compressed Sparse Row format>

In [14]:
x_test = preprocessor.transform(x_test)

In [15]:
print('**Sample train:', sorted(Counter(y_train).items()))

**Sample train: [('N', 418), ('NEU', 133), ('NONE', 139), ('P', 318)]


In [16]:
print('**Sample test:', sorted(Counter(y_test).items()))

**Sample test: [('N', 219), ('NEU', 69), ('NONE', 62), ('P', 156)]


In [17]:
k_fold = ShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

In [18]:
ros_train = RandomOverSampler(random_state=1000)
x_train, y_train = ros_train.fit_resample(x_train, y_train)

In [19]:
print('**OverSample train:', sorted(Counter(y_train).items()))

**OverSample train: [('N', 418), ('NEU', 418), ('NONE', 418), ('P', 418)]


In [20]:
ros_test = RandomOverSampler(random_state=1000)
x_test, y_test = ros_test.fit_resample(x_test, y_test)

In [21]:
print('**OverSample test:', sorted(Counter(y_test).items()))

**OverSample test: [('N', 219), ('NEU', 219), ('NONE', 219), ('P', 219)]


# SVM

In [22]:
clf = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

In [23]:
accuracies_scores = []
recalls_scores = []
precisions_scores = []
f1_scores = []

In [24]:
for train_index, test_index in k_fold.split(x_train, y_train):
    data_train = x_train[train_index]
    target_train = y_train[train_index]
    
    data_test = x_train[test_index]
    target_test = y_train[test_index]

    clf.fit(data_train, target_train)
    predict = clf.predict(data_test)
    # Accuracy
    accuracy = accuracy_score(target_test, predict)
    accuracies_scores.append(accuracy)
    # Recall
    recall = recall_score(target_test, predict, average='macro')
    recalls_scores.append(recall)
    # Precision
    precision = precision_score(target_test, predict, average='weighted')
    precisions_scores.append(precision)
    # F1
    f1 = f1_score(target_test, predict, average='weighted')
    f1_scores.append(f1)

In [25]:
average_recall = round(np.mean(recalls_scores) * 100, 2)
average_precision = round(np.mean(precisions_scores) * 100, 2)
average_f1 = round(np.mean(f1_scores) * 100, 2)
average_accuracy = round(np.mean(accuracies_scores) * 100, 2)

In [26]:
output_result = {'F1-score': average_f1, 'Accuracy': average_accuracy, 'Recall': average_recall, 
                 'Precision': average_precision}

In [27]:
for item, val in output_result.items():
    print('{0} {1}'.format(item, val))

F1-score 66.41
Accuracy 65.48
Recall 66.28
Precision 75.71
